In [ ]:
# 此book仅作为参考，无法直接运行

In [1]:
# Copyright (c) OpenMMLab. All rights reserved.
import argparse
import copy
import os
import os.path as osp
import time

import mmcv
import torch
from mmcv.runner import init_dist
from mmcv.utils import Config, DictAction, get_git_hash

from mmseg import __version__
from mmseg.apis import set_random_seed, train_segmentor
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import collect_env, get_root_logger

/home/znfs/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [2]:
parser = argparse.ArgumentParser(description='Train a segmentor')
parser.add_argument('--config', 
                    type=str,
                    help='train config file path')
parser.add_argument(
    '--work-dir', 
    type=str,
    default='/home/znfs/2021/mmsegmentation/models',
    help='the dir to save logs and models')
parser.add_argument(
    '--load-from', help='the checkpoint file to load weights from')
parser.add_argument(
    '--resume-from', help='the checkpoint file to resume from')
parser.add_argument(
    '--no-validate',
    action='store_true',
    help='whether not to evaluate the checkpoint during training')
group_gpus = parser.add_mutually_exclusive_group()
group_gpus.add_argument(
    '--gpus',
    type=int,
    default=1,
    help='number of gpus to use '
    '(only applicable to non-distributed training)')
group_gpus.add_argument(
    '--gpu-ids',
    type=int,
    nargs='+',
    help='ids of gpus to use '
    '(only applicable to non-distributed training)')
parser.add_argument('--seed', type=int, default=None, help='random seed')
parser.add_argument(
    '--deterministic',
    action='store_true',
    help='whether to set deterministic options for CUDNN backend.')
parser.add_argument(
    '--options', nargs='+', action=DictAction, help='custom options')
parser.add_argument(
    '--launcher',
    choices=['none', 'pytorch', 'slurm', 'mpi'],
    default='none',
    help='job launcher')
parser.add_argument('--local_rank', type=int, default=0)
args = parser.parse_args(args=['--config', '/configs/segformer/segformer_mit-b3_512x512_20k_Drivearea.py']) # 修改此处
if 'LOCAL_RANK' not in os.environ:
    os.environ['LOCAL_RANK'] = str(args.local_rank)


In [46]:
args.config=str('/home/znfs/2021/mmsegmentation/configs/segformer/segformer_mit-b3_512x512_20k_Road.py')
cfg = Config.fromfile(args.config)
if args.options is not None:
    cfg.merge_from_dict(args.options)
# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

# work_dir is determined in this priority: CLI > segment in file > filename
if args.work_dir is not None:
    # update configs according to CLI args if args.work_dir is not None
    cfg.work_dir = args.work_dir
elif cfg.get('work_dir', None) is None:
    # use config filename as default work_dir if cfg.work_dir is None
    cfg.work_dir = osp.join('./work_dirs',
                            osp.splitext(osp.basename(args.config))[0])
if args.load_from is not None:
    cfg.load_from = args.load_from
if args.resume_from is not None:
    cfg.resume_from = args.resume_from
if args.gpu_ids is not None:
    cfg.gpu_ids = args.gpu_ids
else:
    cfg.gpu_ids = range(1) if args.gpus is None else range(args.gpus)

# init distributed env first, since logger depends on the dist info.
if args.launcher == 'none':
    distributed = False
else:
    distributed = True
    init_dist(args.launcher, **cfg.dist_params)
    
# create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# dump config
cfg.dump(osp.join(cfg.work_dir, osp.basename(args.config)))
# init the logger before other steps
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
log_file = osp.join(cfg.work_dir, f'{timestamp}.log')
logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)

# init the meta dict to record some important information such as
# environment info and seed, which will be logged
meta = dict()

# log env info
env_info_dict = collect_env()
env_info = '\n'.join([f'{k}: {v}' for k, v in env_info_dict.items()])
dash_line = '-' * 60 + '\n'
logger.info('Environment info:\n' + dash_line + env_info + '\n' +
            dash_line)
meta['env_info'] = env_info

# log some basic info
logger.info(f'Distributed training: {distributed}')
logger.info(f'Config:\n{cfg.pretty_text}')

# set random seeds
if args.seed is not None:
    logger.info(f'Set random seed to {args.seed}, deterministic: '
                f'{args.deterministic}')
    set_random_seed(args.seed, deterministic=args.deterministic)
cfg.seed = args.seed
meta['seed'] = args.seed
meta['exp_name'] = osp.basename(args.config)

2021-09-09 15:08:08,872 - mmseg - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.8.3 (default, Jul  2 2020, 16:21:59) [GCC 7.3.0]
CUDA available: True
GPU 0: GeForce RTX 2080 Ti
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 10.2, V10.2.89
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.7.1
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.1 Product Build 20200208 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 10.2
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75
  - CuDNN 7.6

In [47]:
model = build_segmentor(
        cfg.model,
        train_cfg=cfg.get('train_cfg'),
        test_cfg=cfg.get('test_cfg'))
model.init_weights()

logger.info(model)

datasets = [build_dataset(cfg.data.train)]
if len(cfg.workflow) == 2:
    val_dataset = copy.deepcopy(cfg.data.val)
    val_dataset.pipeline = cfg.data.train.pipeline
    datasets.append(build_dataset(val_dataset))
if cfg.checkpoint_config is not None:
    # save mmseg version, config file content and class names in
    # checkpoints as meta data
    cfg.checkpoint_config.meta = dict(
        mmseg_version=f'{__version__}+{get_git_hash()[:7]}',
        config=cfg.pretty_text,
        CLASSES=datasets[0].CLASSES,
        PALETTE=datasets[0].PALETTE)
# add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES
# passing checkpoint meta for saving best checkpoint
meta.update(cfg.checkpoint_config.meta)

/home/znfs/2021/mmsegmentation/mmseg/models/backbones/mit.py:315: UserWarning: DeprecationWarning: pretrained is a deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is a deprecated, '
2021-09-09 15:08:11,233 - mmseg - INFO - Use load_from_local loader
2021-09-09 15:08:11,375 - mmseg - INFO - EncoderDecoder(
  (backbone): MixVisionTransformer(
    (layers): ModuleList(
      (0): ModuleList(
        (0): PatchEmbed(
          (projection): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (norm): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
        )
        (1): ModuleList(
          (0): TransformerEncoderLayer(
            (norm1): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
            (attn): EfficientMultiheadAttention(
              (attn): MultiheadAttention(
                (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
              )
              (proj_drop): Dropout(p=

2021-09-09 15:08:11,416 - mmseg - INFO - Loaded 3872 images


In [48]:
print(cfg.data.val)

{'type': 'DriveareaDataset', 'data_root': '/home/znfs/2021/DATASET/DriveArea', 'img_dir': 'img_dir/val', 'ann_dir': 'ann_dir/val', 'pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (1280, 720), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}]}


In [49]:
from mmseg.datasets import build_dataloader



data_loaders = [
        build_dataloader(
            ds,
            cfg.data.samples_per_gpu,
            cfg.data.workers_per_gpu,
            # cfg.gpus will be ignored if distributed
            len(cfg.gpu_ids),
            dist=distributed,
            seed=cfg.seed,
            drop_last=True) for ds in datasets
    ]

In [50]:
for ds in datasets:
    print(ds[1])

{'img_metas': DataContainer({'filename': '/home/znfs/2021/DATASET/DriveArea/img_dir/train/firehydrant_0005.png', 'ori_filename': 'firehydrant_0005.png', 'ori_shape': (720, 1280, 3), 'img_shape': (512, 512, 3), 'pad_shape': (512, 512, 3), 'scale_factor': array([0.8578125 , 0.85833335, 0.8578125 , 0.85833335], dtype=float32), 'flip': True, 'flip_direction': 'horizontal', 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}), 'img': DataContainer(tensor([[[ 0.4851,  0.4851,  0.4851,  ..., -0.5253, -0.5253, -0.6109],
         [ 0.4851,  0.4851,  0.4851,  ..., -0.5253, -0.5253, -0.6109],
         [ 0.4851,  0.4851,  0.4851,  ..., -0.4739, -0.5253, -0.6109],
         ...,
         [-1.8953, -1.9124, -1.8610,  ..., -0.7993, -0.7308, -0.7308],
         [-1.8782, -1.8439, -1.8610,  ..., -0.7308, -0.7137, -0.7479],
         [-1.8610, -1.8439, -1.8439,  ..., -0.6452, -0.6623, -0.7137]],

        [[ 0.66

In [51]:
data = iter(datasets[0])

In [52]:
inputs = next(data)

In [53]:
import numpy as np
temp_seg = inputs['gt_semantic_seg'].data
temp_data = inputs['img'].data
print(type(temp_data))
print(inputs)

<class 'torch.Tensor'>
torch.Size([512, 512])
torch.Size([512, 512])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])
{'img_metas': DataContainer({'filename': '/home/znfs/2021/DATASET/DriveArea/img_dir/train/firehydrant_0004.png', 'ori_filename': 'firehydrant_0004.png', 'ori_shape': (720, 1280, 3), 'img_shape': (512, 512, 3), 'pad_shape': (512, 512, 3), 'scale_factor': array([1.146875 , 1.1472223, 1.146875 , 1.1472223], dtype=float32), 'flip': False, 'flip_direction': 'horizontal', 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}), 'img': DataContainer(tensor([[[-1.4329, -1.3473, -1.3130,  ...,  2.2147,  2.2147,  2.2147],
         [-1.3815, -1.3302, -1.2959,  ...,  2.2147,  2.2147,  2.2147],
         [-1.3473, -1.3987, -1.

In [58]:
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib auto
mean=[123.675, 116.28, 103.53]
std=[58.395, 57.12, 57.375]

r = temp_data[0].data.numpy()*std[0]+mean[0]
g = temp_data[1].data.numpy()*std[1]+mean[1]
b = temp_data[2].data.numpy()*std[2]+mean[2]

img=np.array([r,g,b]) #合成三通道图像， 此时数组的尺寸是3*224*224，我们需要的是224*224*3
img=img.transpose(1,2,0).astype(np.uint8) 
print(img.shape)
# img = Image.fromarray(temp_data[0].data.numpy().astype('uint8')).convert('RGB')
plt.imshow(img)
plt.show()

Using matplotlib backend: Qt5Agg
(512, 512, 3)


In [57]:

plt.ion()
img = Image.fromarray(temp_seg[0].data.numpy().astype('uint8')).convert('RGB')
plt.imshow(img)
plt.show()

In [15]:
print(datasets[0].CLASSES)
print(datasets[0].PALETTE)
print(datasets)

('drivearea', 'obstacle', 'other')
[[1, 1, 1], [2, 2, 2], [0, 0, 0]]


In [59]:
for ds in datasets:
    print(ds)